
# Access DB2 Warehouse on Cloud and explore the data with Python

This notebook shows how to access a DB2 data warehouse or DB2 database when using Python. The examples use a DB2 warehouse, but the instructions apply to both DB2 Warehouse and DB2.



## Table of contents

1. [Info](#Info)
1. [Create Version](#Version)
1. [Setup](#Setup) 
1. [Import the *ibmdbpy* Python library](#Import-the-ibmdbpy-Python-library)
1. [Insert the database connection credentials](#Insert-the-database-connection-credentials)
1. [Create the database connection](#Create-the-database-connection)
1. [Use dataframe to read and manipulate tables](#Use-dataframe-to-read-and-manipulate-tables)
1. [Use SQL](#Use-SQL)
1. [Explore the trafficking data](#Explore-the-trafficking-data)
1. [Close the database connection](#Close-the-database-connection)
1. [Share the notebook](#Share)
1. [Save to GitHub](#GitHub)
1. [Help](#Help)
1. [Summary](#Summary)

<a id="Info"></a>
## Info
Select the Info icon to display general information about the notebook. <br>
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/info-button.png" ><br>
You can change the notebook name or add a description here.  
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/bleonardb3/DSX/master/Lab-1/images/Lab1Information.png" >

<a id="Version"></a>
## Create Version 

Save a version of the notebook by selecting File > Save Version 
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/FileOptions.PNG" > or by selecting the Versions icon. <img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/versions-button.png" ><br>
You can have up to ten (10) versions of a notebook.   Notebook versions are saved in a FIFO manner.

<a id="Setup"></a>
## Setup

Before beginning you will need a *DB2 Warehouse on Cloud* instance. DB2 Warehouse is a fully managed cloud data warehouse, purpose-built for analytics. It offers massively parallel processing (MPP) scale and compatibility with a wide range of business intelligence (BI) tools.  

1. __RIGHT-CLICK__ [this link](https://github.com/bleonardb3/DSX/raw/master/Lab-1/data/Female-human-trafficking.csv) and select __Save Link As...__ to save the `Female-human-trafficking.csv` file to your desktop.
1. Download the [DB2 Warehouse setup instructions](https://github.com/bleonardb3/DSX/blob/master/Lab-1/DB2Warehouse_DSXPOT.pdf).  You need to download the PDF in order for the links to work.
1. Open up the PDF file that you've just downloaded in the previous step and follow the instructions there to setup DB2 Warehouse, load the human trafficking data and create a DSX data connection.
1. Click on the icon below to navigate to the DB2 Warehouse provisioning screen. 

<a class="ibm-tooltip" href="https://console.ng.bluemix.net/catalog/services/dashdb" target="_blank" title="" id="ibm-tooltip-0">
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/DB2Warehouse.png" >
</a>

<a id="Import-the-ibmdbpy-Python-library"></a>
## Import the *ibmdbpy* Python library

Python support for DB Warehouse and DB2 is provided by the [ibmdbpy Python library](https://pypi.python.org/pypi/ibmdbpy). Connecting to DB2 Warehouse or DB2 is also enabled by a DB2 driver, libdb2.so.  The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM DB2 Warehouse and IBM DB2. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing.

### Setup a *JDBC* connection
The JDBC Connection is based on a Java virtual machine. From the ibmdbpy library you can use JDBC to connect to a remote DB2 Warehouse/DB2 instance. To be able to use JDBC to connect, we need to import the *JayDeBeApi* package and we need to add the DB2 Warehouse/DB2 data connector jar file to the java class path.

To run a cell, click on the cell and click the run button in the toolbar or press __SHIFT-ENTER__.

Run the following commands to install and load the `JayDeBeApi` package and the `ibmdbpy` library into your notebook:

In [6]:
import jaydebeapi
from ibmdbpy import IdaDataBase
from ibmdbpy import IdaDataFrame

In [7]:
import pixiedust

In [9]:
# connection jar for DB2 Warehouse on Cloud -- you may have to restart the kernel and re-run cells.   If you do, a warning will appear.

pixiedust.installPackage("file:///usr/local/src/data-connectors-1.4.1/db2jcc4-10.5.0.6.jar")

Package already installed: file:///usr/local/src/data-connectors-1.4.1/db2jcc4-10.5.0.6.jar


# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IBM_API_KEY_ID': 'u6PqCrHsrK32WSyOrXWgZyLX3DhtSgeToPZLat7ajC53',
    'IAM_SERVICE_ID': 'iam-ServiceId-db5c3ffc-eef6-4cfc-a41b-c93b0382e8cd',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'dsxlabs2bb36949eb3140ac9e3cbb1a26190518',
    'FILE': 'Female-human-trafficking.csv'
}
<a id="Insert-the-database-connection-credentials"></a>
## Insert the database connection credentials

Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, find your database connection and click the __Insert to code__ link under the connection name to have a `credential
# @hidden_cell
credentials_2 = {
  'database':'BLUDB',
  'password':"""532da0445039""",
  'sg_service_url':'https://sgmanager.ng.bluemix.net',
  'host':'dashdb-entry-yp-dal09-07.services.dal.bluemix.net',
  'username':'dash13303'
}
s_1` dictionary added to the notebook.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/connections-button.png" >

__Note__: When this dictionary is inserted, it might have a different name than `credentials_1`.  If so change the dictionary name to `credentials_1` so that the rest of the notebook will work.

Connecting to DB2 Warehouse requires the following information which are provided by the credentials dictionary inserted:
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID
* User password

The information `credentials_1` will be used to build a connection string in a subsequent step.

In [17]:
# insert connection here
credentials_1 = {
  'database':'BLUDB',
  'password':"""532da0445039""",
  'sg_service_url':'https://sgmanager.ng.bluemix.net',
  'host':'dashdb-entry-yp-dal09-07.services.dal.bluemix.net',
  'username':'dash13303'
}

<a id="Create-the-database-connection"></a>
## Create the database connection

The following code snippet creates a connection string `connection_string`
and uses the `connection_string` to create a database connection object:


In [18]:
connection_string='DASHDB;Database={database};Hostname={host};Port=50000;PROTOCOL=TCPIP;UID={username};PWD={password}'.format(**credentials_1)
idadb = IdaDataBase(dsn=connection_string)

<a id="Use-dataframe-to-read-and-manipulate-tables"></a>
## Use a dataframe to read and manipulate tables

You can now use the connection object `idadb` to query the database:

In [19]:
df = idadb.show_tables(show_all = True)
df.head(10)

,TABSCHEMA,TABNAME,OWNER,TYPE
0,GOSALES,BRANCH,DB2INST1,T
1,GOSALES,CONVERSION_RATE,DB2INST1,T
2,GOSALES,COUNTRY,DB2INST1,T
3,GOSALES,CURRENCY_LOOKUP,DB2INST1,T
4,GOSALES,EURO_CONVERSION,DB2INST1,T
5,GOSALES,INVENTORY_LEVELS,DB2INST1,T
6,GOSALES,ORDER_DETAILS,DB2INST1,T
7,GOSALES,ORDER_HEADER,DB2INST1,T
8,GOSALES,ORDER_METHOD,DB2INST1,T
9,GOSALES,PRODUCT,DB2INST1,T


In [20]:
idadb.exists_table_or_view('GOSALESDW.EMP_EXPENSE_FACT')

True

Using our previously opened IdaDataBase instance named ‘idadb’, we can open one or several IdaDataFrame objects. They behave like pointers to remote tables.

Let us open the *EMP_EXPENSE_FACT* data set, assuming it is stored in the database under the name ‘GOSALESDW.EMP_EXPENSE_FACT’. The following cell assigns the dataset to a IdaDataFrame.   IdaDataFrame copies the Pandas interface for DataFrame objects to ensure intuitive interaction for end-users.

The [Pandas data analysis library](http://pandas.pydata.org/) provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas allows easy processing and manipulation of tabular data, so it is a perfect fit for data extracted from relational databases.


In [26]:
idadf1 = IdaDataFrame(idadb, 'GOSALESDW.EMP_EXPENSE_FACT')
idadf1.head()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20100131,11106,43638,4011,2131,8049,165.000,3378.21
1,20100131,11106,43638,4011,2122,8054,0.110,554.72
2,20100131,11106,43638,4011,2120,8052,0.080,403.44
3,20100131,11106,43638,4011,2130,8050,0.005,1664.70
4,20100131,11106,43638,4011,2124,8056,0.030,151.29



from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_bcbe474539bb440b98ee4167bc6494cc = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp-dal09-07.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=dash13303;PWD=532da0445039')

ida_df_1 = IdaDataFrame(idadb_bcbe474539bb440b98ee4167bc6494cc, 'GOSALESDW.EMP_EXPENSE_FACT')
ida_df_1.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_bcbe474539bb440b98ee4167bc6494cc.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/
### Automatic creation of dataframes for tables<br>
<div class="panel-group" id="accordion-1">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-1" href="#collapse1-1">Optional Advanced:</a></h4>
    </div>
    <div id="collapse1-1" class="panel-collapse collapse">
      <div class="panel-body">
Try adding the IdaDataFrame for the table in question from the Connections view -- do all the following commands still work?<br><br>
To add the dataframe, select the "Find and Add Data" icon (the one with 0 and 1s), select Connections, then "Insert to code" for the dataframe.    You will have to select the schema and table to use.

      </div>
    </div>
  </div>
</div>

In [27]:
# Try adding the dataframe here -- be sure to rename it to idadf so the following cells will still work
#insert to code on connections tab then create IdaDataFrame
from ibmdbpy import IdaDataBase, IdaDataFrame
idadb_bcbe474539bb440b98ee4167bc6494cc = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp-dal09-07.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=dash13303;PWD=532da0445039')

idadf = IdaDataFrame(idadb_bcbe474539bb440b98ee4167bc6494cc, 'GOSALESDW.EMP_EXPENSE_FACT')
idadf.head()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20100131,11106,43638,4011,2131,8049,165.000,3378.21
1,20100131,11106,43638,4011,2122,8054,0.110,554.72
2,20100131,11106,43638,4011,2120,8052,0.080,403.44
3,20100131,11106,43638,4011,2130,8050,0.005,1664.70
4,20100131,11106,43638,4011,2124,8056,0.030,151.29


In [31]:
idadf.dtypes

,TYPENAME
DAY_KEY,INTEGER
ORGANIZATION_KEY,INTEGER
POSITION_KEY,INTEGER
EMPLOYEE_KEY,INTEGER
EXPENSE_TYPE_KEY,INTEGER
ACCOUNT_KEY,INTEGER
EXPENSE_UNIT_QUANTITY,DOUBLE
EXPENSE_TOTAL,DECIMAL


You can very easily explore the data in the IdaDataFrame by using built in functions.

Use IdaDataFrame.head to get the first n records of your data set (default 5):

In [32]:
idadf.head()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20100131,11142,43611,4809,2120,8052,0.08,233.33
1,20100131,11143,43625,4563,2120,8052,0.08,168.67
2,20100131,11143,43625,4563,2104,8050,30.00,383.33
3,20100131,11142,43611,4809,2104,8050,15.00,265.15
4,20100131,11143,43625,4563,2122,8054,0.11,231.92


Use IdaDataFrame.tail to get the last n records of your data set (default 5) -- try running this multiple times:

In [33]:
idadf.tail(10)

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
127974,20130731,11169,43638,4125,2120,8052,0.080,791.68
127975,20130731,11169,43638,4125,2122,8054,0.110,1088.56
127976,20130731,11169,43638,4125,2124,8056,0.030,296.88
127977,20130731,11169,43638,4125,2130,8050,0.005,6057.97
127978,20130731,11169,43638,4125,2131,8049,157.500,3664.39
127979,20130731,11168,43639,4127,2120,8052,0.080,492.11
127980,20130731,11168,43639,4127,2122,8054,0.110,676.65
127981,20130731,11168,43639,4127,2124,8056,0.030,184.54
127982,20130731,11168,43639,4127,2130,8050,0.005,1795.57
127983,20130731,11168,43639,4127,2131,8049,165.000,4355.77


__Note__: Because DB2 Warehouse operates on a distributed system, the order of rows using IdaDataFrame.head and IdaDataFrame.tail is not guaranteed unless the table is sorted (using an ‘ORDER BY’ clause) or a column is declared as index for the IdaDataFrame (parameter/attribute indexer).

IdaDataFrame also implements most attributes that are available in a pandas DataFrame, here is an example of getting the _shape_ of the data frame (rows, columns):

In [34]:
idadf.shape

(127984, 8)

In [35]:
idadf.columns

Index(['DAY_KEY', 'ORGANIZATION_KEY', 'POSITION_KEY', 'EMPLOYEE_KEY',
       'EXPENSE_TYPE_KEY', 'ACCOUNT_KEY', 'EXPENSE_UNIT_QUANTITY',
       'EXPENSE_TOTAL'],
      dtype='object')

Several standard statistics functions from the pandas interface are also available for IdaDataFrame.  You can use the __TAB__ key after the `.` to see the methods and attributes available in an object in a pulldown menu. 

For example, let us calculate various summary statistics for the data set by using the `describe()` method for the IdaDataFrame instance:

In [37]:
idadf.describe()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
count,1.279840e+05,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000,127984.000000
mean,2.011514e+07,11157.820524,43627.441797,4489.672811,2120.315289,8052.091636,31.291244,1318.107960
std,1.036554e+04,31.272648,12.175158,298.987628,9.386336,2.532779,57.717638,2078.720318
min,2.010013e+07,11101.000000,43601.000000,4001.000000,2101.000000,8049.000000,-15.000000,-658.010000
25%,2.011013e+07,11131.000000,43617.000000,4216.000000,2120.000000,8050.000000,0.080000,211.280000
50%,2.011123e+07,11166.000000,43630.000000,4480.000000,2122.000000,8052.000000,0.110000,438.045000
75%,2.012103e+07,11182.000000,43637.000000,4763.000000,2124.000000,8054.000000,16.250000,1598.345000
max,2.013073e+07,11204.000000,43656.000000,4972.000000,2137.000000,8056.000000,172.500000,28776.920000


__Note__: It is possible to subset the rows of an IdaDataFrame by accessing the IdaDataFrame with a slice object. You can also use the IdaDataFrame.loc attribute, which contains an ibmdbpy.Loc object. However, the row selection might be inaccurate if the current IdaDataFrame is not sorted or does not contain an indexer. This is due to the fact that DB2 Warehouse stores the data across several nodes if available. Moreover, because DB2 Warehouse is by default a column oriented database, row numbers are undefined:

In [38]:
idadf_new = idadf[0:9] # Select the first 10 rows
idadf_new.head(10)

/usr/local/src/conda3_runtime.v27/4.1.1/lib/python3.5/site-packages/ibmdbpy/indexing.py:118: UserWarning: Row order is not guaranteed if no indexer was given and the dataset was not sorted
  " was given and the dataset was not sorted")


,day_key,organization_key,position_key,employee_key,expense_type_key,account_key,expense_unit_quantity,expense_total
0,20100531,11191,43650,4959,2122,8054,0.11,1159.23
1,20100531,11191,43650,4959,2131,8049,157.50,10538.46
2,20100531,11191,43650,4959,2124,8056,0.03,316.15
3,20100630,11191,43650,4959,2120,8052,0.08,843.08
4,20100630,11191,43650,4959,2131,8049,165.00,10538.46
5,20100630,11191,43650,4959,2124,8056,0.03,316.15
6,20100630,11191,43650,4959,2122,8054,0.11,1159.23
7,20100731,11191,43650,4959,2120,8052,0.08,843.08
8,20100731,11191,43650,4959,2124,8056,0.03,316.15
9,20100731,11191,43650,4959,2122,8054,0.11,1159.23


<a id="Use-SQL"></a>
## Use SQL
We can also use SQL to get the top 10.

In [39]:
top_10_df = idadb.ida_query('SELECT * FROM GOSALESDW.EMP_EXPENSE_FACT LIMIT 10')
top_10_df

,day_key,organization_key,position_key,employee_key,expense_type_key,account_key,expense_unit_quantity,expense_total
0,20120731,11167,43636,4192,2122,8054,0.11,620.16
1,20120731,11167,43636,4192,2124,8056,0.03,169.13
2,20120731,11167,43636,4192,2131,8049,157.50,5637.82
3,20120831,11167,43636,4192,2120,8052,0.08,451.03
4,20120831,11167,43636,4192,2122,8054,0.11,620.16
5,20120831,11167,43636,4192,2124,8056,0.03,169.13
6,20120831,11167,43636,4192,2131,8049,172.50,5637.82
7,20120930,11167,43636,4192,2120,8052,0.08,451.03
8,20120930,11167,43636,4192,2122,8054,0.11,620.16
9,20120930,11167,43636,4192,2124,8056,0.03,169.13


Find the top 10 employees (by id) by the number of transactions in the `GOSALESDW.EMP_EXPENSE_FACT` table.

In [42]:
idadb.ida_query('SELECT EMPLOYEE_KEY, COUNT(*) AS COUNT FROM GOSALESDW.EMP_EXPENSE_FACT GROUP BY EMPLOYEE_KEY ORDER BY COUNT DESC LIMIT 10')

,employee_key,count
0,4112,263.0
1,4134,263.0
2,4135,261.0
3,4019,260.0
4,4368,260.0
5,4053,260.0
6,4016,260.0
7,4129,259.0
8,4480,259.0
9,4020,258.0


<a id="Explore-the-trafficking-data"></a>
## Explore the trafficking data
__Time to apply what you've learned!__

Complete the tasks that are commented in the code blocks.  If you get stuck, check out the hints.  If you're still stuck, read the solution.  If you have any questions, feel free to ask.

### Create an ibmdbpy dataframe called `trafficking_df` and set it to the name of the trafficking table in your DB2 Warehouse <br>

<div class="panel-group" id="accordion-2">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-2" href="#collapse2-1">Hint 1</a></h4>
    </div>
    <div id="collapse2-1" class="panel-collapse collapse">
      <div class="panel-body">     
Use the Connections table to insert the dataframe.   Be sure to name the result trafficking_df!
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-2" href="#collapse2-2">Solution</a></h4>
    </div>
    <div id="collapse2-2" class="panel-collapse collapse">
      <div class="panel-body">     
Select the data cell you wish the code to be added to<br>
Select the Find and Add Data icon<br>
Select Connections<br>
Select Insert to Code for the connection you wish to use (there should only be one)<br>
Select Insert ibmdbpy IdaDataframe<br>
Select the Schema you used to load the data (this schema should start with DASH)<br>
Select the Table (there should only be one)<br>
Select Insert Code<br>
Rename the result to trafficking_df
      </div>
    </div>
  </div>
</div>

In [53]:
# The code was removed by DSX for sharing.

,INTERNAL_ID,VETTING_LEVEL,DESCRIPTION,VETTING_LEVEL_PREDICTION,ALGORITHM_NAME,NAME,GENDER,BIRTH_DATE,BIRTH_COUNTRY,BIRTH_COUNTRY_CODE,...,ARRIVAL_AIRPORT_COUNTRY_CODE,ARRIVAL_AIRPORT_IATA,ARRIVAL_AIRPORT_MUNICIPALITY,ARRIVAL_AIRPORT_REGION,DEPARTURE_AIRPORT_COUNTRY_CODE,DEPARTURE_AIRPORT_IATA,DEPARTURE_AIRPORT_MUNICIPALITY,DEPARTURE_AIRPORT_REGION,UUID,AGE
0,1,100,NA,30,DECISION_TREE,Kathleen Bailey,F,1985-11-27,Ghana,GH,...,US,BMI,Bloomington/Normal,US-IL,AE,AUH,Abu Dhabi,AE-AZ,81d28245-a6d4-40e3-bac0-df61cc6625ce,31
1,2,30,NA,30,DECISION_TREE,Kelly Robbins,F,1971-01-17,Pakistan,PK,...,US,CLE,Cleveland,US-OH,UA,KBP,Kiev,UA-32,b4d817b3-2154-4351-b0e3-bcd736a76bf5,46
2,3,100,NA,30,DECISION_TREE,Linda Stewart,F,1997-04-15,Ghana,GH,...,US,TPA,Tampa,US-FL,RU,KJA,Krasnoyarsk,RU-KYA,a52a6477-1f5e-4b09-a5e3-f76333a7eb10,19
3,4,100,NA,30,DECISION_TREE,Stacey Courtney Gonzalez,F,1984-03-27,Ghana,GH,...,US,TOL,Toledo,US-OH,RU,AER,Sochi,RU-KDA,5fcbbf15-8268-4306-ac26-406c2faae251,32
4,5,30,NA,30,DECISION_TREE,Erika Patie Fowler,F,1998-06-18,Ghana,GH,...,US,FWA,Fort Wayne,US-IN,IE,ORK,Cork,IE-C,6d1fe59a-6828-4bc5-9ded-2194aa238461,18


In [54]:
idadb_bcbe474539bb440b98ee4167bc6494cc.exists_table_or_view(trafficking_df.name)

# The output of this block should be:
# True

True

In [55]:
print(trafficking_df.shape)
trafficking_df.dtypes

(1085, 28)


,TYPENAME
INTERNAL_ID,SMALLINT
VETTING_LEVEL,SMALLINT
DESCRIPTION,VARCHAR
VETTING_LEVEL_PREDICTION,SMALLINT
ALGORITHM_NAME,VARCHAR
NAME,VARCHAR
GENDER,VARCHAR
BIRTH_DATE,DATE
BIRTH_COUNTRY,VARCHAR
BIRTH_COUNTRY_CODE,VARCHAR


### Show the top 5 records from the table using a method of the IdaDataFrame <br>
<div class="panel-group" id="accordion-12">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-12" href="#collapse1-12">Hint 1</a></h4>
    </div>
    <div id="collapse1-12" class="panel-collapse collapse">
      <div class="panel-body">
      
Heads or tails?
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-12" href="#collapse2-12">Solution</a></h4>
    </div>
    <div id="collapse2-12" class="panel-collapse collapse">
      <div class="panel-body">
      Heads!<br>
<pre>
trafficking_df.head()
</pre> 
      </div>
    </div>
  </div>
</div>


In [57]:
# enter answer here
trafficking_df.head(5)

,INTERNAL_ID,VETTING_LEVEL,DESCRIPTION,VETTING_LEVEL_PREDICTION,ALGORITHM_NAME,NAME,GENDER,BIRTH_DATE,BIRTH_COUNTRY,BIRTH_COUNTRY_CODE,...,ARRIVAL_AIRPORT_COUNTRY_CODE,ARRIVAL_AIRPORT_IATA,ARRIVAL_AIRPORT_MUNICIPALITY,ARRIVAL_AIRPORT_REGION,DEPARTURE_AIRPORT_COUNTRY_CODE,DEPARTURE_AIRPORT_IATA,DEPARTURE_AIRPORT_MUNICIPALITY,DEPARTURE_AIRPORT_REGION,UUID,AGE
0,1,100,NA,30,DECISION_TREE,Kathleen Bailey,F,1985-11-27,Ghana,GH,...,US,BMI,Bloomington/Normal,US-IL,AE,AUH,Abu Dhabi,AE-AZ,81d28245-a6d4-40e3-bac0-df61cc6625ce,31
1,2,30,NA,30,DECISION_TREE,Kelly Robbins,F,1971-01-17,Pakistan,PK,...,US,CLE,Cleveland,US-OH,UA,KBP,Kiev,UA-32,b4d817b3-2154-4351-b0e3-bcd736a76bf5,46
2,3,100,NA,30,DECISION_TREE,Linda Stewart,F,1997-04-15,Ghana,GH,...,US,TPA,Tampa,US-FL,RU,KJA,Krasnoyarsk,RU-KYA,a52a6477-1f5e-4b09-a5e3-f76333a7eb10,19
3,4,100,NA,30,DECISION_TREE,Stacey Courtney Gonzalez,F,1984-03-27,Ghana,GH,...,US,TOL,Toledo,US-OH,RU,AER,Sochi,RU-KDA,5fcbbf15-8268-4306-ac26-406c2faae251,32
4,5,30,NA,30,DECISION_TREE,Erika Patie Fowler,F,1998-06-18,Ghana,GH,...,US,FWA,Fort Wayne,US-IN,IE,ORK,Cork,IE-C,6d1fe59a-6828-4bc5-9ded-2194aa238461,18


### Query the trafficking table so that we get the vetting_level and the number of times that vetting_level appears in the trafficking table, ordered by the vetting_level<br>
<div class="panel-group" id="accordion-13">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse1-13">Hint 1</a></h4>
    </div>
    <div id="collapse1-13" class="panel-collapse collapse">
      <div class="panel-body">
      
It is similar to:

<pre>
<resource>.ida_query('SELECT EMPLOYEE_KEY, COUNT(&#42;) AS COUNT FROM GOSALESDW.EMP_EXPENSE_FACT GROUP BY EMPLOYEE_KEY ORDER BY COUNT DESC LIMIT 10')
</pre>
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse2-13">Hint 2</a></h4>
    </div>
    <div id="collapse2-13" class="panel-collapse collapse">
      <div class="panel-body">
      
The SQL query is:

<pre>
'SELECT VETTING_LEVEL, COUNT(&#42;) AS COUNT FROM ' + trafficking_table + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL'
</pre>
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse3-13">Solution 1</a></h4>
    </div>
    <div id="collapse3-13" class="panel-collapse collapse">
      <div class="panel-body">
      
<pre>
idadb.ida_query('SELECT VETTING_LEVEL, COUNT(&#42;) AS COUNT FROM ' + trafficking_df.name + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL')
</pre>

      </div>
    </div>
  </div>
    <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse3-14">Solution 2</a></h4>
    </div>
    <div id="collapse3-14" class="panel-collapse collapse">
      <div class="panel-body">
      
<pre>
trafficking_df.ida_query('SELECT VETTING_LEVEL, COUNT(&#42;) AS COUNT FROM ' + trafficking_df.name + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL')
</pre>

      </div>
    </div>
  </div>
</div>


In [62]:
# Enter query here
db.ida_query('SELECT VETTING_LEVEL, COUNT(*) AS COUNT FROM FEMALE_HUMAN_TRAFFICKING GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL')

,vetting_level,count
0,10,42.0
1,20,40.0
2,30,96.0
3,100,907.0


<a id="Close-the-database-connection"></a>
## Close the database connection

To ensure expected behaviors, IdaDataBase instances need to be closed.

__Note__: Closing the *IdaDataBase* is equivalent to closing the connection: once the connection is closed, it is no longer possible to use the *IdaDataBase* instance and any *IdaDataFrame* instances that were opened on this connection.

In [63]:
idadb.close()

Connection closed.


<a id="Share"></a>
## Share the notebook
You can share a notebook by selecting the Share icon.<br>
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/share-button.png" ><br>
This displays a Share Notebook popup which gives the user a link which can can be sent to other users.   Anyone with the link will see the most recent version of the notebook.   The user can determine what level of notebook information will be shared.
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Share-Window.PNG" ><br>

<a id="GitHub"></a>
## Publish the notebook to gist/GitHub

Select the Github/Gist integration icon from the command bar (it looks like an open box with an arrow pointing up).<br>
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/github-button.png" ><br>

If you wish to publish to a gist, nothing more is needed.   However, if you want to publish to github, you will need a security token.

Select Publish on GitHub<br>
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/NoGitHub.PNG" ><br>
You will get a popup stating that you need to set a token in your account settings.   This can be reached by selecting the account settings link in the popup or by selecting your personal icon (upper right hand corner) and selecting settings.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Profile.PNG" ><br>
In the Profile settings page select Integrations and paste your github token there.   

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Integrations.PNG" ><br>
I have provided one for you should you wish to try this which uses the account at http://github.com/dsxspare3/DSX-Demo

I broke it into two pieces because otherwise github will delete it when publishing to github because it is a valid token.   Paste them together when putting in the github token in DSX.

Part 1: 8e8ff3b15d92438fc4  Part 2: a2a0aad611310d2562d8b5

Finally, you need to add the URL for the GitHub in the Settings section of your project.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/ProjectSettings.PNG" ><br>

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/GitHubURL.PNG" ><br>

You should now be able to publish to GitHub for this project.   When publishing, you are asked for the path to save.   Since all users will be using the same GitHub I suggest publishing to a folder with your username.

<a id="Help"></a>
## Get Help

Select the icon in the lower right corner 
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Help-Icon.PNG"/>
to display a help window connecting to IBM DSX Support.
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/DSX/master/Lab-1/images/Help-Window.PNG"/>

<a id="Summary"></a>
## Summary

This notebook demonstrated how to establish a JDBC connection to a DB2 Warehouse (or DB2) database from Python, work with data frames and explore the data using SQL and data frame object attributes and methods.

## Take it farther
### Free courses on <a href="https://cognitiveclass.ai/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu/" rel="noopener noreferrer" target="_blank">Cognitive Class.ai</a>: <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu" rel="noopener noreferrer" target="_blank"><img src = "https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png" width=600px> </a>